In [3]:
import pandas as pd
from datetime import datetime, timedelta
import os

In [2]:
spot_df = pd.read_csv("./ArchivosRendimientos/Benchmarks/Historico_SPOT.csv")
spot_df.rename(columns={"FECHA":"Fecha", "PRECIO SUCIO":"Spot"}, inplace=True)
spot_df = spot_df[["Fecha", "Spot"]]
spot_df["Fecha"] = pd.to_datetime(spot_df["Fecha"])
spot_df

,Fecha,Spot
0,2024-01-02,17.0440
1,2024-01-03,17.0364
2,2024-01-04,17.0348
3,2024-01-05,16.8832
4,2024-01-08,16.8100
...,...,...
300,2025-03-07,20.2672
301,2025-03-10,20.3623
302,2025-03-11,20.2582
303,2025-03-12,20.1797


In [3]:
precios_bmks_isimp_df = pd.read_excel("./ArchivosPeergroups/Benchmarks/Historico_ISIMP.xlsx", skiprows=2, skipfooter=4)
precios_bmks_isimp_df["Fecha"] = pd.to_datetime(precios_bmks_isimp_df["Fecha"], format="%d/%m/%Y")
precios_bmks_isimp_df.rename(columns={"Índice":"ISIMP"}, inplace=True)
precios_bmks_isimp_df = precios_bmks_isimp_df[["Fecha", "ISIMP"]]
precios_bmks_isimp_df

,Fecha,ISIMP
0,2024-01-02,260.585246
1,2024-01-03,260.588146
2,2024-01-04,260.656244
3,2024-01-05,260.673049
4,2024-01-08,260.905944
...,...,...
296,2025-03-07,291.468154
297,2025-03-10,291.634970
298,2025-03-11,291.702763
299,2025-03-12,291.773025


In [4]:
precios_bmks_acwi_df = pd.read_csv("./ArchivosPeergroups/Benchmarks/Historico_ACWI.csv")
precios_bmks_acwi_df.rename(columns={"FECHA":"Fecha", "PRECIO SUCIO":"ACWI"}, inplace=True)
precios_bmks_acwi_df["Fecha"] = pd.to_datetime(precios_bmks_acwi_df["Fecha"], format="%Y-%m-%d")
precios_bmks_acwi_df = precios_bmks_acwi_df[["Fecha", "ACWI"]]
precios_bmks_acwi_df

,Fecha,ACWI
0,2024-01-02,1719.739600
1,2024-01-03,1705.343640
2,2024-01-04,1703.309652
3,2024-01-05,1689.670656
4,2024-01-08,1702.516800
...,...,...
300,2025-03-07,2421.930400
301,2025-03-10,2370.375343
302,2025-03-11,2345.899560
303,2025-03-12,2350.733253


In [8]:
precios_bmks_valmer_df = pd.read_csv("./ArchivosPeergroups/Benchmarks/Benchmarks_SP_Historico_MD.csv")
precios_bmks_valmer_df.rename(columns={"FECHA":"Fecha"}, inplace=True)
precios_bmks_valmer_df["Fecha"] = pd.to_datetime(precios_bmks_valmer_df["Fecha"], format="%Y%m%d")
precios_bmks_valmer_df.drop(columns=[col for col in precios_bmks_valmer_df.columns if "Unnamed" in col], inplace=True)
precios_bmks_valmer_df.set_index("Fecha", inplace=True)
precios_bmks_valmer_df

,S&P/BMV Government MBONOS 5-10 Year Bond Index,S&P/BMV Government MBONOS 1-5 Year Bond Index,S&P/BMV Government CETES Bond Index,S&P/BMV Government Inflation-Linked UDIBONOS 1+ Year Bond Index,S&P/BMV Mexico Sovereign Bond Index,S&P/BMV Mexico Sovereign Inflation-Linked Bond Index,S&P/BMV Sovereign MBONOS Bond Index,S&P/BMV Sovereign MBONOS 1-3 Year Bond Index,S&P/BMV Sovereign MBONOS 3-5 Year Bond Index,S&P/BMV Sovereign MBONOS 10-20 Year Bond Index,...,S&P/BMV Sovereign UDIBONOS & CBICS Bond Index,S&P/BMV Sovereign UDIBONOS & CBICS 1-3 Year Bond Index,S&P/BMV Sovereign UDIBONOS & CBICS 3-5 Year Bond Index,S&P/BMV Sovereign UDIBONOS & CBICS 5-10 Year Bond Index,S&P/BMV Sovereign UDIBONOS & CBICS 10+ Year Bond Index,S&P/BMV Sovereign International UMS 5-10 Year Target Maturity 30% Capped Bond Index (MXN),S&P/BMV Sovereign BONDESF Bond Index (MXN),S&P/BMV Sovereign BONDES Bond Index (MXN),S&P/BMV Sovereign International UMS 5-10 Year Target Maturity 30% Capped Bond Index (USD),S&P/BMV Green Social & Sustainable Target Duration Bond Index (MXN)
Fecha,,,,,,,,,,,,,,,,,,,,,
2001-01-04,NaN,NaN,NaN,NaN,NaN,NaN,100.000000,100.000000,100.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-05,NaN,NaN,NaN,NaN,NaN,NaN,100.092145,100.090847,100.093364,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-08,NaN,NaN,NaN,NaN,NaN,NaN,100.277668,100.325108,100.233323,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-09,NaN,NaN,NaN,NaN,NaN,NaN,100.390324,100.480925,100.305671,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-10,NaN,NaN,NaN,NaN,NaN,NaN,100.310827,100.491887,100.141794,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-07,494.171129,436.812220,398.230100,510.326162,449.001594,493.051046,771.591870,656.562229,797.044142,328.336458,...,137.931759,154.012112,144.564503,142.838602,127.179754,133.978840,140.023744,140.009830,122.686301,159.619967
2025-03-10,493.545160,437.081818,398.521394,510.870043,448.788226,493.572850,770.857436,657.172515,797.175835,327.382786,...,138.080240,154.237078,144.822728,143.120338,127.242343,134.138825,140.135766,140.121913,122.777112,159.641775
2025-03-11,492.509318,437.250243,398.672409,510.958887,448.847727,493.667112,770.922666,657.557922,797.250822,327.665437,...,138.110771,154.479999,144.809636,143.331202,127.176562,134.191050,140.175076,140.161201,122.543498,159.561361


In [ ]:
propiedades_fondos_path = "./ArchivosRendimientos/PropiedadesFondos"
propiedades_fondos_filename = os.listdir(propiedades_fondos_path)[0]
propiedades_fondos_df = pd.read_excel(os.path.join(propiedades_fondos_path, propiedades_fondos_filename))
fondo2tipo_fondo = {fondo:propiedades_fondos_df[propiedades_fondos_df["Fondo"] == fondo]["Tipo"].item() for fondo in propiedades_fondos_df["Fondo"]}

{'VECTRF': 'RF',
 'VECTPRE': 'RF',
 'VECTMD': 'RF',
 'VECTFI': 'RF',
 'VECTCOR': 'RF',
 'VECTTR': 'RF',
 'VECTPZO': 'RF',
 'VECTCOB': 'RV',
 'INCOME': 'RF',
 'VECTIND': 'RV',
 'VECTPA': 'RV',
 'VECTMIX': 'RV',
 'VECTSIC': 'RV',
 'VECTUSD': 'RV',
 'COMMODQ': 'RV',
 'VECTUSA': 'RV',
 'MXRATES': 'RV',
 'EQUITY': 'RV',
 'NEXTGEN': 'RV',
 'BALANCE': 'RV',
 'DYNAMIC': 'RV'}

In [ ]:
hoy = datetime.today()
hoy = datetime(year=hoy.year, month=hoy.month, day=hoy.day)
hoy

datetime.datetime(2025, 3, 20, 0, 0)

In [ ]:
hoy - timedelta(days=180)

datetime.datetime(2024, 9, 21, 0, 0)

In [15]:
(hoy - timedelta(days=1)) - timedelta(days=180)

datetime.datetime(2024, 9, 20, 0, 0)

In [17]:
datetime(year=2025, month=3, day=18) - timedelta(days=180)

datetime.datetime(2024, 9, 19, 0, 0)

In [32]:
datetime(year=2025, month=3, day=21) - datetime(year=2025, month=2, day=28)

datetime.timedelta(days=21)

In [34]:
(3.274532/3.2564 - 1)#*(360/21)

0.005568112025549743

In [3]:
import streamlit as st
import yfinance as yf
from datetime import datetime
import pandas as pd
from pandas.tseries.offsets import CustomBusinessDay
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import calendar
import exchange_calendars as xcals
import numpy as np
import os
import time
import io
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

def formatea_columna_tabla_rendimientos_MiVector(col):
    temp_col = col.copy()
    if temp_col.dtype == "O":
        temp_col = temp_col.replace("N[/]D[\%]", "", regex=True)
        temp_col = temp_col.replace("%|,", "", regex=True)
        temp_col = temp_col.replace("", np.nan, regex=True)

        if temp_col.isna().sum() != len(temp_col):
            if not temp_col.str.contains(r"[A-Za-z]").any():
                temp_col = temp_col.astype(float)
                temp_col = temp_col/100 if col.name not in ["Precio", "(+)Títulos en Circulación"] else temp_col

    return temp_col

def descarga_rendimientos_MiVector(fechas):
    chrome_options = Options()
    chrome_options.add_argument("--headless=new")
    driver = webdriver.Chrome(options=chrome_options)
    start_url = "https://www.vectoronline.com.mx/servicios/fondos/consultas/precios_rendimientos.php"
    driver.get(start_url)

    rendimientos_MiVector_fechas_df = pd.DataFrame()
    for fecha_dt in fechas:
        fecha_str = fecha_dt.strftime("%Y-%m-%d")
        time.sleep(3)

        driver.find_element(by=By.ID, value="dpFecha").clear()
        driver.find_element(by=By.ID, value="dpFecha").send_keys(fecha_str)
        driver.find_element(by=By.ID, value="btnMostrar").click()

        time.sleep(3)

        tables = pd.read_html(io.StringIO(driver.page_source))

        rendimientos_MiVector_df = []
        for i in range(len(tables)):
            if i % 2 != 0:
                col_names = tables[i - 1].columns
                temp_df = tables[i].rename(columns={i:col_names[i] for i in range(len(col_names))})
                temp_records = temp_df.to_dict(orient="records")
                rendimientos_MiVector_df.extend(temp_records)

        rendimientos_MiVector_df = pd.DataFrame.from_records(rendimientos_MiVector_df)
        rendimientos_MiVector_df["Fecha"] = fecha_str
        rendimientos_MiVector_fechas_df = pd.concat([rendimientos_MiVector_fechas_df, rendimientos_MiVector_df], axis=0, ignore_index=True)

    rendimientos_MiVector_fechas_df["Fecha"] = pd.to_datetime(rendimientos_MiVector_fechas_df["Fecha"], format="%Y-%m-%d")
    rendimientos_MiVector_fechas_df = rendimientos_MiVector_fechas_df.apply(lambda col: formatea_columna_tabla_rendimientos_MiVector(col), axis=0)

    driver.close()

    return rendimientos_MiVector_fechas_df

propiedades_fondos_path = "./ArchivosRendimientos/PropiedadesFondos"
propiedades_fondos_filename = os.listdir(propiedades_fondos_path)[0]
propiedades_fondos_df = pd.read_excel(os.path.join(propiedades_fondos_path, propiedades_fondos_filename))

precios_fondos_valmer_path = "./ArchivosPeergroups/PreciosFondosValmer"
precios_fondos_valmer_filename = os.listdir(precios_fondos_valmer_path)[0]
precios_fondos_valmer_df = pd.read_csv(os.path.join(precios_fondos_valmer_path, precios_fondos_valmer_filename))
precios_fondos_valmer_df = precios_fondos_valmer_df[["FECHA", "EMISORA", "SERIE", "PRECIO SUCIO"]]
precios_fondos_valmer_df.rename(columns={"FECHA":"Fecha", "EMISORA":"Fondo", "SERIE":"Serie", "PRECIO SUCIO":"Precio"}, inplace=True)
precios_fondos_valmer_df["Fecha"] = pd.to_datetime(precios_fondos_valmer_df["Fecha"], format="%Y-%m-%d")

precios_fondos_df = precios_fondos_valmer_df.copy()
precios_fondos_df = precios_fondos_df[
    pd.Series(list(zip(precios_fondos_df["Fondo"], precios_fondos_df["Serie"]))).isin(list(zip(propiedades_fondos_df["Fondo"], propiedades_fondos_df["Serie"])))
].reset_index(drop=True)

fechas = [datetime.today()]
temp_precios_fondos_df = descarga_rendimientos_MiVector(fechas)
temp_precios_fondos_df = temp_precios_fondos_df[
    pd.Series(list(zip(temp_precios_fondos_df["Fondo"], temp_precios_fondos_df["Serie"]))).isin(list(zip(propiedades_fondos_df["Fondo"], propiedades_fondos_df["Serie"])))
].reset_index(drop=True)
temp_precios_fondos_df = temp_precios_fondos_df[["Fecha", "Fondo", "Serie", "Precio"]]

precios_fondos_df = pd.concat([precios_fondos_df, temp_precios_fondos_df], axis=0, ignore_index=True)
precios_fondos_df = precios_fondos_df[["Fecha", "Fondo", "Precio"]].pivot(index="Fecha", columns="Fondo")
precios_fondos_df = precios_fondos_df.droplevel(level=0, axis=1)
precios_fondos_df.columns.name = None
precios_fondos_df

,BALANCE,COMMODQ,DYNAMIC,EQUITY,INCOME,MXRATES,NEXTGEN,VECTCOB,VECTCOR,VECTFI,...,VECTMD,VECTMIX,VECTPA,VECTPRE,VECTPZO,VECTRF,VECTSIC,VECTTR,VECTUSA,VECTUSD
Fecha,,,,,,,,,,,,,,,,,,,,,
2024-01-02,1.663866,1.225499,1.856247,1.282715,0.771363,1.264668,1.285670,7.983501,2.251985,29.141187,...,23.652331,1.507268,2.862019,2.966595,17.566639,2.866247,1.452843,1.561223,1.675532,1.900161
2024-01-03,1.659027,1.232383,1.853236,1.280499,0.768894,1.265088,1.269067,8.019913,2.251275,29.091732,...,23.658423,1.501150,2.843895,2.967220,17.515709,2.867152,1.448921,1.557873,1.671446,1.909495
2024-01-04,1.651513,1.239797,1.853013,1.269823,0.768597,1.265491,1.247903,8.016326,2.251719,29.096797,...,23.667247,1.478570,2.782505,2.968179,17.511902,2.868052,1.437313,1.555436,1.657321,1.909130
2024-01-05,1.647033,1.225495,1.850075,1.267453,0.766158,1.266310,1.239472,8.015562,2.251043,29.054857,...,23.674929,1.472142,2.768468,2.968841,17.462763,2.868949,1.435300,1.547613,1.651479,1.909631
2024-01-08,1.650564,1.216393,1.851994,1.258549,0.767379,1.266594,1.226939,7.944219,2.254932,29.096493,...,23.696830,1.475624,2.803327,2.971524,17.495451,2.871643,1.424473,1.550336,1.638455,1.893025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-21,1.919258,1.471333,2.031656,1.752204,0.835874,1.441704,1.629141,9.465640,2.607420,34.028516,...,27.070779,1.782737,2.775600,3.392491,19.968872,3.274532,1.975925,1.742695,2.341117,2.398586
2025-03-24,1.918765,1.466378,2.039167,1.757752,0.833952,1.442176,1.632137,9.508972,2.605715,33.971245,...,27.090472,1.780083,2.753046,3.395055,19.843199,3.277126,1.981831,1.743079,2.354593,2.410187
2025-03-25,1.920819,1.464756,2.051416,1.767100,0.833386,1.442437,1.642180,9.423445,2.605273,33.960836,...,27.096427,1.786368,2.752521,3.395847,19.819228,3.277993,1.991091,1.741636,2.372491,2.389418


In [4]:
fechas = [datetime.today()]
temp_precios_fondos_df = descarga_rendimientos_MiVector(fechas)
temp_precios_fondos_df = temp_precios_fondos_df[
    pd.Series(list(zip(temp_precios_fondos_df["Fondo"], temp_precios_fondos_df["Serie"]))).isin(list(zip(propiedades_fondos_df["Fondo"], propiedades_fondos_df["Serie"])))
].reset_index(drop=True)
temp_precios_fondos_df = temp_precios_fondos_df[["Fecha", "Fondo", "Serie", "Precio"]]
temp_precios_fondos_df

,Fecha,Fondo,Serie,Precio
0,2025-03-28,BALANCE,XD,1.932210
1,2025-03-28,DYNAMIC,X,2.053985
2,2025-03-28,VECTRF,XF0,3.280594
3,2025-03-28,VECTPRE,XF0,3.398955
4,2025-03-28,VECTMD,XF0,27.121306
5,2025-03-28,VECTFI,XF0,34.069352
6,2025-03-28,VECTCOR,XF0,2.611896
7,2025-03-28,VECTTR,XF0,1.748131
8,2025-03-28,VECTPZO,XF0,19.937663
9,2025-03-28,VECTCOB,XF,9.538953


In [1]:
0.00031777773546992627*0.4

0.0001271110941879705